# [E-04] 작사가 인공지능 만들기 


### 목표 : 적절한 노래 가사가 나오는 모델 만들기 
## 1. 데이터 전처리
    - 데이터 가져오기
    - 데이터 정제
    - 토큰화 및 패딩
    - 데이터셋 분리 

## [2. 모델 정하고 학습하기](#5.-모델-정하고-학습하기)
    - 하이퍼파라미터 2개(embedding, hidden_size)만 조절해서 최적의 모델 찾기
    - LSTM 사용 
    - Loss 함수 : SparseCategoricalCrossentropy
    - dropout 추가 
    - epochs 10으로 고정
   - ### [모델 학습하기](#5-1.-모델-학습하기)
   
   
## 3. 모델 평가하기
## [4. 하이퍼파라미터 수정 과정 및 회고](#7.-하이퍼파라미터-수정-과정-및-회고)
## [5. 결론](#8.-결론) 
<br>
<br>

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 일단 문서 내부 링크 이동 만드는 방법

- 링크 넣는 것과 방식은 같음 [링크 걸고 싶은 글](#"이동하고자 하는 위치의 글쓰기, 이때 빈칸은 - 처리")
- 예시 **[2. 모델 정하고 학습하기](#5.-모델-정하고-학습하 기)**  <- 위에 2번 링크 할 떄 어떻게 했는지 보여주기 위해 일부러 하 기로 빈칸 넣음
- 이동할 위치의 글이 영어라면 반드시 소문자만 가능!

## 1. 데이터 다운로드 
- 폴더 안에 여러 텍스트 파일 준비 
- shell에 아래의 코드 입력해서 연결

```python
mkdir -p ~/aiffel/lyricist/models
ln -s ~/data ~/aiffel/lyricist/data
```

## 2. 데이터 읽어오기 

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'  # * 모든  
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 넣음 
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth']


## 3. 데이터 정제

- preprocess_sentence() 함수 이용
- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거
- 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 



In [2]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1 소문자 변경, 양쪽 공백 제거
    sentence = re.sub(r"(\(.*\))", '', sentence) #2 ()단어는 불필요하므로 삭제
    sentence = re.sub(r"(\[.*\])", '', sentence) #3 [] 단어는 불필요하므로 삭제 
    sentence = re.sub(r"([?.!])", r" \1 ", sentence) #4 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) #5 여러 개 공백 하나로
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence) #6 제시된 것 아닌 모든 문자를 하나의 공백
    sentence = sentence.strip() #7 양쪽 공백 제거
    sentence = '<start> ' + sentence + ' <end>' #8 문장 앞 뒤로 추가 
    return sentence

print(preprocess_sentence("It's ~~~a good @#$~ time   (hello)  [hi]"))

<start> it s a good time <end>


### 정제해서 corpus에 넣기

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 : continue    # 문장 길이가 0이면 넘어가고
    
    
    # 정제를 하고 넣기
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 15개 이상 건너뛰기 
    if len(preprocessed_sentence.split()) > 15 : continue
    
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 10개 확인
#corpus[100:110]

### 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용

In [4]:
import tensorflow as tf

#12000 단어 사용

def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)     # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tensor = tokenizer.texts_to_sequences(corpus)     # tokenizer를 이용해 corpus를 Tensor로 변환
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춤, 문장 앞은 padding='pre'를 사용
    # 토큰의 개수는 최대 15개인데 데이터셋 분리할 때 start하나를 빼므로 16으로 잡음
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 16)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor.shape

[[   2   48    4 ...    0    0    0]
 [   2   14 2992 ...    0    0    0]
 [   2   32    6 ...    0    0    0]
 ...
 [   2    4  109 ...    0    0    0]
 [   2  256  191 ...    0    0    0]
 [   2    6   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc4280c7b50>


(161139, 16)

In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : the
6 : you
7 : and
8 : a
9 : to
10 : it


## 4. 데이터셋 분리

- 훈련 데이터와 평가 데이터를 분리
- tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리
- 단어장의 크기는 12,000 이상 으로 설정, 총 데이터의 20% 를 평가 데이터셋으로 사용

In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.

src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   48    4   92  303   61   52    8  960 5812    3    0    0    0
    0]
[  48    4   92  303   61   52    8  960 5812    3    0    0    0    0
    0]


In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, 
                                                          test_size = 0.2, random_state = 7)

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (128911, 15)
Target Train: (128911, 15)


- 데이터셋 객체 생성, 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성하는 방법을 흔히 사용

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

## 5. 모델 정하고 학습하기 
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계
- Loss는 아래 제시된 Loss 함수를 그대로 사용

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.dropout(out)
        out = self.linear(out)
        
        return out

## 하이퍼파라미터 조절하기 
- Dense representation은 각각의 속성을 독립적인 차원으로 나타내지 않는다. 대신, 우리가 정한 개수의 차원으로 대상을 대응시켜서 표현, 즉! **특징을 몇 개나 잡아서 볼거냐?**

In [11]:
# 워드 벡터의 차원 수, 
embedding_size = 1024

# hidden layer 수 
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[ 3.31113843e-04,  9.81116464e-05, -2.98146508e-04, ...,
         -1.15414659e-04, -6.21682208e-04, -2.11040213e-04],
        [ 8.59663181e-04, -3.36788362e-04, -2.65131734e-04, ...,
          1.13698057e-04, -6.49792433e-04, -2.81940214e-04],
        [ 1.22446415e-03, -3.53877491e-04, -2.35360174e-04, ...,
          7.73117179e-04, -7.54277280e-04, -4.49173589e-04],
        ...,
        [ 1.91593624e-03,  2.41484563e-03, -4.89657791e-03, ...,
          3.92926187e-04,  1.31762458e-03, -5.28598379e-04],
        [ 1.91294297e-03,  2.73081753e-03, -6.01140969e-03, ...,
          2.46376731e-04,  1.78548112e-03, -6.30152877e-04],
        [ 1.83439360e-03,  2.94462568e-03, -7.00923568e-03, ...,
          6.50704460e-05,  2.24087574e-03, -7.40077521e-04]],

       [[ 3.31113843e-04,  9.81116464e-05, -2.98146508e-04, ...,
         -1.15414659e-04, -6.21682208e-04, -2.11040213e-04],
        [ 9.99651616e-04, -8.24580493e-05, -1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  8392704   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 41,375,457
Trainable params: 41,375,457
Non-trainable params: 0
_________________________________________________________________


## 5-1. 모델 학습하기 

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

In [15]:
model.fit(enc_train, dec_train, epochs=10, validation_data =(enc_val, dec_val))

Epoch 1/10
4029/4029 [==============================] - 192s 47ms/step - loss: 2.8089 - val_loss: 2.5782
Epoch 2/10
4029/4029 [==============================] - 188s 47ms/step - loss: 2.4992 - val_loss: 2.4157
Epoch 3/10
4029/4029 [==============================] - 189s 47ms/step - loss: 2.3116 - val_loss: 2.3057
Epoch 4/10
4029/4029 [==============================] - 188s 47ms/step - loss: 2.1456 - val_loss: 2.2296
Epoch 5/10
4029/4029 [==============================] - 188s 47ms/step - loss: 1.9976 - val_loss: 2.1717
Epoch 6/10
4029/4029 [==============================] - 188s 47ms/step - loss: 1.8716 - val_loss: 2.1403
Epoch 7/10
4029/4029 [==============================] - 187s 47ms/step - loss: 1.7640 - val_loss: 2.1183
Epoch 8/10
4029/4029 [==============================] - 187s 46ms/step - loss: 1.6759 - val_loss: 2.1094
Epoch 9/10
4029/4029 [==============================] - 187s 46ms/step - loss: 1.6027 - val_loss: 2.0953
Epoch 10/10
4029/4029 [==============================] 

## 6. 평가하기 

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len = 15 ):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [25]:
generate_text(model, tokenizer, init_sentence="<start> you")

# '<start> you re the only one who knows that <end> '
#'<start> i m gonna make it alright but not right now <end> '

'<start> you re the only one i want <end> '

## 7. 하이퍼파라미터 수정 과정 및 회고 
<br>

### 1. 그냥 별 생각없이 돌려봤을 때 
(embedding_size , hidden_size, epochs)
1. (256, 1024, 10) --> val_loss: 2.5968  /  첫번째 에폭 loss: 3.2777 - val_loss: 2.9878

hidden_size를 두배로 늘려봄 <br> 
2. (256, 2048, 10) --> val_loss: 2.5664 

embedding_size를 2배로 늘려봄 <br>
3. (512, 1024, 10) --> val_loss: 2.4830  /  첫번째 loss: 3.0280 - val_loss: 2.7192

- 1번은 기억이 잘 안나는데.. 2번, 3번의 경우 loss값은 계속 떨어지지만, val_loss값은 떨어지다가 epoch이 증가할수록 다시 증가하는 추세로 나왔다. 과적합 되는 것을 볼 수 있었다.

- '<start> you re the only one who knows that <end> '
- '<start> i m gonna make it alright but not right now <end> '
    
=======================================================================================================================

### 2. 하이퍼파라미터에 대해 알아보고 돌리기(이때까지도 잘못 이해했음)  
- **그냥 무작정 하이퍼파라미터를 바꾸니 의미가 없다....** 한번 학습시키면 시간이 오래 걸려서 하이퍼파라미터를 막 변경하기가 어려웠다. 막 넣으면 안되지 암 그렇고 말고
- 분명히 embedding size와 hidden size만 바꾸면 좋은 결과값이 나온다는데 왜 이 두 하이퍼파라미터가 선택되었는지 궁금해졌다. 
- embedding, hidden size에 대해 정확하게 파악한 후 모델을 돌려보기로 했다.
    - 관련링크 : https://dreamgonfly.github.io/blog/word2vec-explained/

#### embedding_size는 워드 벡터의 차원 수인데 15개로 세팅했으므로 15로 고정하고 hidden_size를 변경해보기로 결정 !

4. (15, 2048, 10) --> val_loss: 2.4427 

5. (15, 1024, 10) --> val_loss: 2.5059  hidden_size를 줄이니 val_loss값 증가 

6. (15, 3000, 10) --> 돌려놓고 잤는데 에폭 10 돌리다가 끊김 에폭 9의 val_loss: 2.4137 에폭 7부터 증가 
    
======================================================================================================================
 
### 3. 제대로 알고 돌리기     
<span style = 'background-color: #fff5b1'> **embedding size란 특징들을 얼마나 볼 것인가 라는 뜻, 속성 갯수!**</span> **이해하도록 도와주신 은서 퍼실님 감사합니다!!**
- 이전에 생각은 embedding size가 워드 벡터의 차원의 수 라고 해서 토큰화를 15개 했으니 그 하나씩을 한 차원으로 보고 임베딩을 15로 넣었는데 그것은 단지 입력값인 것이라서 임베딩을 15로 했다는 것은 그 값들을 고려할 특징을 딸랑 15개만 본다는 의미였다. 
- 우리가 조절해야 하는 것은 얼만큼의 특징들을 고려하고 은닉층을 조절하면서(각종 하이퍼 파라미터들) 최적의 모델을 만들 것인가이다 
- 그래서 다시 하이퍼파라미터들을 위와 같이 고려해본 후 세팅해보았다. 
    
7. (1024, 512, 10) --> 101s 23ms/step - loss: 1.7245 - val_loss: 2.4628
    - 특징을 1024개로 많이 보고 은닉층을 512개로 좀 줄여서 모델 복잡도를 떨어뜨려보았다. val_loss값은 2.2를 못 맞추었지만 두번째로 낮은 val_loss값이 나왔고 학습하는데 걸리는 시간도 위의 경우 1 에폭 당 400s 이상의 시간이 걸렸는데 이 경우 101s만 걸리는 것을 볼 수 있다. 
    - '<start> you marianne baddest once once lifeline lifeline lifeline lifeline lifeline lifeline contempt contempt four ' 7번 결과값 
    - hidden_size를 줄이니 의미없이 반복되는 단어들이 너무 많이 나왔다. 학습 시간은 짧아졌지만 쓸 수 없는 모델, hidden size는 너무 낮추면 안되겠다! 512 이상으로 돌리기!

    
8. (1024, 1024, 10) --> 195s 44ms/step - loss: 1.1571 - val_loss: 2.3809
    - '<start> you re the one that i adore you <end> '
    - hidden size를 올리니 의미없이 반복되는 단어가 사라지는 것을 볼 수 있음

    
9. (2048, 1024, 10) --> 239s 54ms/step - loss: 1.1770 - val_loss: 2.3844
    - 특징을 좀 더 높여서 학습시켜봄
    - '<start> you know that i love you <end> ' 적절한 노래 가사, 이거랑 똑같은 노래 제목이 있음!

    
10. (1024, 2048, 10) -->  498s 113ms/step - loss: 0.9763 - val_loss: 2.3531
    - val_loss가 내려가기는 하는데.... 
    - 2.1 이하로 내려가다가 다시 올라가서 결국 2.2가 넘음

=================================================================================================================

### 4. dropout 넣어야겠다!!
    - val_loss값이 중간에 계속 증가하는 추세가 보여서 도저히 안되겠다. dropout 추가
    
            
11. <span style="color:red"> (512, 2048, 10) --> 508s 115ms/step - loss: 1.4068 - val_loss: 2.1124 
    - '<start> you re the one that makes me strong <end> '
    - 적절한 노래 가사에 val_loss값도 2.2 이하 
    - 학습 시간이 1 epochs 당 508s로 오래 걸림 </span>
    

12. 9번째도 val_loss값이 낮게 나왔고, 글 내용도 괜찮게 나왔기 때문에 9번 하이퍼파라미터 세팅(2048, 1024, 10) + dropout 해보기 
    - 11번과 비교해서 어떤게 더 나을까?
    - 249s 56ms/step - loss: 1.7439 - val_loss: 2.2412
    - '<start> you know i m bad i m bad you know it <end> '
    - 가사 단어가 다양하지 않고 반복적으로 나옴 별로 좋지 않은 모델
    
    
13. (1024, 1024, 10) --> 205s 47ms/step - loss: 1.6499 - val_loss: 2.2001
    - '<start> you know i m bad i m bad you know it <end> '
    - 가사 단어가 다양하지 않고 반복적으로 나옴 별로 좋지 않은 모델
    

======================================================================================================================

### 5. 학습데이터 조금 더 전처리 해보기
- 성돈님께서 알려주신 단어 15개 이상은 사용하지 않는 코드 추가, 성돈님 알려주셔서 감사합니다!! 
- if len(preprocessed_sentence.split()) > 15 : continue
- 위의 경우는 단어 갯수 15개로 그냥 잘라버리는 것이므로 완성되지 못한 문장을 학습에 사용하기 때문에 아예 빼버리는게 나음

- 11번이 잘 나오긴 했는데 hidden_size가 2048로 너무 커서 적당한 모델(1024, 1024)로 돌려봄
- 13번 모델 평가 시 단어가 반복적이였는데 전처리를 더 하고 나서는 어떻게 될지 궁금했음 

14. <span style="color:red">(1024, 1024, 10) --> 187s 46ms/step - loss: 1.5436 - val_loss: 2.0969
    - '<start> you re the only one i want <end> '
    - 적절한 노래 가사, 다른 단어로 바꿔봐도 괜찮은 것을 볼 수 있음 
    - epochs 당 187s로 학습 시간도 적당
    - **13번과 동일한 조건에서 학습 데이터 전처리가 더 잘 되어서 val_loss 값이 줄어든 것을 볼 수 있다**

    


# 8. 결론 
## 1. 데이터 전처리를 잘해야 한다. garbage in garbage out
## 2. 하이퍼파라미터에 대해 제대로 이해하지 못한다면 오랫동안 헤맬 수 있다. 는 나!!! 
## 3. embedding, hidden_size = 1024 / val_loss: 2.0969